# Testing the paper's roi detection method.


In [5]:
import cv2
from imutils import face_utils
import dlib
import face_recognition as fr
import numpy as np
p = "shape_predictor_68_face_landmarks.dat"
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(p)

# Getting the image
image = cv2.imread("WIN_20210226_11_48_56_Pro.jpg")

# Converting the image to gray scale
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

face_locations = fr.face_locations(image)
face_encodings = fr.face_encodings(image, face_locations)

for (top, right, bottom, left), name in zip(face_locations, face_encodings):
    x,y,w,h = cv2.rectangle(image, (left, top), (right, bottom), (0, 0, 255), 2)

# Get faces into image
rects = detector(gray, 0)
mask = np.zeros(image.shape, dtype=np.uint8)
mask = cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY)
# For each detected face, find the landmark.
for (i, rect) in enumerate(rects):
    # Make the prediction and transform it to numpy array
    shape = predictor(gray, rect)
    shape = face_utils.shape_to_np(shape)
    # Draw on our image, all the finded cordinate points (x,y)
    for (x, y) in shape:
        ROI = cv2.bitwise_and(image, mask)
        cv2.circle(image, (x, y), 2, (0, 255, 0), -1)
        x_new,y_new,w_new,h_new = cv2.boundingRect(mask)
        result = ROI[y_new:y_new+h_new,x_new:x_new+w_new]
        mask = mask[y_new:y_new+h_new,x_new:x_new+w_new]
        result[mask==0] = (255,255,255)
        cv2.imshow('result', result)
        cv2.waitKey()
# Show the image
cv2.imshow("crop", crop_img)
k = cv2.waitKey(5) & 0xFF

KeyboardInterrupt: 

In [ ]:
import cv2
from imutils import face_utils
import dlib
import face_recognition as fr
import numpy as np
p = "shape_predictor_68_face_landmarks.dat"
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(p)

# Getting the image
image = cv2.imread("WIN_20210226_11_48_56_Pro.jpg")
mask = np.zeros(image.shape, dtype=np.uint8)
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
rects = detector(gray, 0)
for (i, rect) in enumerate(rects):
    # Make the prediction and transform it to numpy array
    shape = predictor(gray, rect)
    shape = face_utils.shape_to_np(shape)
    for (x, y) in shape:
        cv2.circle(mask, (x,y), 110, (255,255,255), -1)
        #cv2.circle(mask, (x, y), 2, (0, 255, 0), -1)
        # Bitwise-and for ROI
        ROI = cv2.bitwise_and(image, mask)

        # Crop mask and turn background white
        mask = cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY)
        x,y,w,h = cv2.boundingRect(mask)
        result = ROI[y:y+h,x:x+w]
        mask = mask[y:y+h,x:x+w]
        result[mask==0] = (255,255,255)

        cv2.imshow('result', result)
        cv2.waitKey()


In [6]:
import cv2
from imutils import face_utils
import dlib
import face_recognition as fr
import numpy as np
from matplotlib import pyplot as plt

# Load in color image for face detection
image = cv2.imread('WIN_20210227_10_32_53_Pro.jpg')

# Convert the image to RGB colorspace
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

# Make a copy of the original image to draw face detections on
image_copy = np.copy(image)

# Convert the image to gray
gray_image = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)

# Detect faces in the image using pre-trained face dectector
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
faces = face_cascade.detectMultiScale(gray_image, 1.25, 6)

# Print number of faces found
print('Number of faces detected:', len(faces))

# Get the bounding box for each detected face
face_crop = []
for f in faces:
    x, y, w, h = [ v for v in f ]
    cv2.rectangle(image_copy, (x,y), (x+w, y+h), (255,0,0), 3)
    # Define the region of interest in the image
    face_crop.append(gray_image[y:y+h, x:x+w])

# Display the face crops
for face in face_crop:
    cv2.imshow('face',face)
    cv2.waitKey(0)

Number of faces detected: 1


# Forehead detection

In [ ]:
# import the necessary packages
from imutils import face_utils
import numpy as np
import imutils
import dlib
import cv2

# initialize dlib's face detector (HOG-based) and then create
# the facial landmark predictor
p = "shape_predictor_68_face_landmarks.dat"
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(p)
# load the input image, resize it, and convert it to grayscale
image = cv2.imread('WIN_20210227_10_32_53_Pro.jpg')
image = imutils.resize(image, width=500)
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
# detect faces in the grayscale image
rects = detector(gray, 1)

# loop over the face detections
for (i, rect) in enumerate(rects):
    # determine the facial landmarks for the face region, then
    # convert the landmark (x, y)-coordinates to a NumPy array
    shape = predictor(gray, rect)
    shape = face_utils.shape_to_np(shape)
    # loop over the face parts individually
    for (name, (i, j)) in face_utils.FACIAL_LANDMARKS_IDXS.items():
        # clone the original image so we can draw on it, then
        # display the name of the face part on the image
        clone = image.copy()
        cv2.putText(clone, name, (10, 30), cv2.FONT_HERSHEY_SIMPLEX,
                    0.7, (0, 0, 255), 2)
        # loop over the subset of facial landmarks, drawing the specific face part
        #counter=0
        for (x, y) in shape[i:j]:
            cv2.circle(clone, (x, y), 1, (0, 0, 255), -1)# extract the ROI of the face region as a separate image
            (x, y, w, h) = cv2.boundingRect(np.array([shape[i:j]]))
            roi = image[y:y + h, x:x + w]
            roi = imutils.resize(roi, width=250, inter=cv2.INTER_CUBIC)
            # show the particular face part
            cv2.imshow("ROI", roi)
            cv2.imshow("Image", clone)
            cv2.waitKey(0)
            #counter+=1

        # visualize all facial landmarks with a transparent overlay
        output = face_utils.visualize_facial_landmarks(image, shape)
        #cv2.imshow("Image", output)
        #cv2.waitKey(0)

# Extract Forehead ;-)



In [ ]:
import face_recognition, PIL.Image, PIL.ImageDraw,math
import numpy as np

frame = "WIN_20210227_10_32_53_Pro.jpg"

image = face_recognition.load_image_file(frame) # read image.
face_locations = face_recognition.face_locations(image) # detects all the faces in image
face_landmarks_list = face_recognition.face_landmarks(image)

for face_location in face_locations:
    img = PIL.Image.open(frame) 
    top,right,bottom,left = face_location 
    diff = math.floor((top - bottom) * 0.3) # calc the 30 percent of the face len.
    right_eyebrow_landmarks = np.asarray(face_landmarks_list[0]['right_eyebrow']) # right eyebrow points.
    rbottom = right_eyebrow_landmarks[-1] # the most right point according to x.
    rmin = right_eyebrow_landmarks.min(axis = 0)[1]
    
    left_eyebrow_landmarks = np.asarray(face_landmarks_list[0]['left_eyebrow'])
    lbottom= left_eyebrow_landmarks[0] # the most left point according to x.
    lmin = left_eyebrow_landmarks.min(axis = 0)[1]
    
    bottom = min(lmin,rmin).item(0) # bottom point of the forehead.
    
    bottom = bottom - (0.02 * bottom) # improve bottom location by 2 percent.
    
    img = img.crop((lbottom[0], lbottom[1]+diff, rbottom[0],bottom)) # adding diff to top to make the forehead bigger.
    img.save("forehead.jpg") 
